In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas
import numpy as np
import time
pd.set_option('display.max_columns',75)

In [2]:
nearest_subways_df = pd.read_csv('data/nearest_subways_df.csv',index_col=0)
rent_estimate_df = pd.read_csv('data/rent_estimate_df.csv',index_col=0)
rent_estimate_df=rent_estimate_df.rename(columns = {'addr':'address'})

In [3]:
median_rents = pd.read_csv('../datasets/Neighborhood_MedianRentalPrice_AllHomes.csv')
median_rents_ny = median_rents[(median_rents.CountyName=='New York')| (median_rents.CountyName=='Kings')]
median_rents_ny = median_rents_ny.rename(columns={'RegionName':'neighborhood'})


In [4]:
median_rents_ny = median_rents_ny.drop(columns=['2010-02', '2010-03', '2010-04', '2010-05','2010-06', '2010-07', '2010-08', '2010-09','2010-10', '2010-11', '2010-12', '2011-01', '2011-02', '2011-03',
       '2011-04', '2011-05', '2011-06', '2011-07', '2011-08', '2011-09','2011-10', '2011-11', '2011-12','City','State','Metro','SizeRank'])

median_rents_ny.neighborhood = median_rents_ny.neighborhood.str.lower()
median_rents_ny = median_rents_ny.reset_index(drop=True)

## create multi-indexed dataframe to store neighborhood median rental values by month

In [5]:
med_rents = median_rents_ny.set_index('neighborhood')
med_rents.columns = pd.MultiIndex.from_tuples([tuple(c.split('-')) for c in med_rents.columns])
med_rents = med_rents.fillna(0)

In [6]:
med_rents.head()

CountyName    2012                                          \
                          NaN      01      02      03      04      05      06   
neighborhood                                                                    
upper west side      New York  3275.5  3399.0  3200.0  3200.0  3312.0  3400.0   
upper east side      New York  2775.0  2800.0  2700.0  2700.0  2750.0  2700.0   
washington heights   New York  1550.0  1500.0  1500.0  1500.0  1500.0  1551.5   
williamsburg            Kings  2800.0  2750.0  2750.0  2750.0  2800.0  2800.0   
bedford stuyvesant      Kings  1600.0  1540.0  1600.0  1600.0  1700.0  1700.0   

                                                                      2013  \
                        07      08      09      10      11      12      01   
neighborhood                                                                 
upper west side     3366.5  3400.0  3400.0  3450.0  3400.0  3312.5  3300.0   
upper east side     2700.0  2750.0  2795.0  2800.0  2800.0  2800.0  2800.0   
washington heights  1550.0  1600.0  1600.0  1595.0  1600.0  1600.0  1595.0   
williamsburg        2800.0  2850.0  3200.0  3200.0  3100.0  3099.0  3150.0   
bedford stuyvesant  1705.0  1750.0  1750.0  1750.0  1750.0  1750.0  1800.0   

                                                                            \
                        02      03      04      05      06      07      08   
neighborhood                                                                 
upper west side     3300.0  3300.0  3295.0  3299.0  3250.0  3200.0  3250.0   
upper east side     2800.0  2750.0  2795.0  2700.0  2600.0  2500.0  2600.0   
washington heights  1600.0  1675.0  1695.0  1750.0  1750.0  1775.0  1750.0   
williamsburg        3060.0  3000.0  3000.0  3149.0  3150.0  3095.0  3092.0   
bedford stuyvesant  1800.0  1850.0  1850.0  1995.0  2000.0  2100.0  2250.0   

                                                      2014                  \
                        09      10      11      12      01      02      03   
neighborhood                                                                 
upper west side     3495.0  3495.5  3500.0  3500.0  3500.0  3500.0  3505.0   
upper east side     2800.0  2995.0  3140.0  3100.0  3095.0  3095.0  3100.0   
washington heights  1675.0  1675.0  1650.0  1675.0  1650.0  1650.0  1675.0   
williamsburg        3156.0  3400.0  3225.0  3200.0  3200.0  3200.0  3200.0   
bedford stuyvesant  2200.0  2200.0  2150.0  2100.0  2100.0  2000.0  2000.0   

                                                                            \
                        04      05      06      07      08      09      10   
neighborhood                                                                 
upper west side     3595.0  3550.0  3680.0  3600.0  3600.0  3600.0  3700.0   
upper east side     3195.0  3100.0  3170.0  3097.5  3200.0  3200.0  3195.0   
washington heights  1695.0  1700.0  1700.0  1700.0  1687.5  1700.0  1700.0   
williamsburg        3200.0  3200.0  3340.0  3395.0  3411.0  3400.0  3400.0   
bedford stuyvesant  2100.0  2150.0  2200.0  2300.0  2300.0  2300.0  2400.0   

                                     ...      2015                          \
                        11      12   ...        07      08      09      10   
neighborhood                         ...                                     
upper west side     3657.5  3600.0   ...    3695.0  3675.0  3700.0  3650.0   
upper east side     3150.0  3170.0   ...    3200.0  3200.0  3200.0  3200.0   
washington heights  1687.5  1700.0   ...    1700.0  1700.0  1700.0  1700.0   
williamsburg        3312.5  3312.5   ...    3500.0  3500.0  3450.0  3495.0   
bedford stuyvesant  2400.0  2395.0   ...    2400.0  2415.0  2450.0  2400.0   

                                      2016                                  \
                        11      12      01      02      03      04      05   
neighborhood                                                                 
upper west 

In [7]:
rent_estimate_df = rent_estimate_df.reset_index(drop=True)

In [8]:
def get_med_prices(df):
    start = time.time()
    median_nb_rents = []
    for i, row in df.iterrows():
        if (i+1)%5000 == 0:
            print('Runtime: {} seconds, {} iterations remaining'.format(round(time.time()-start,2),len(df)-i))

        #if neighborhood median value exists
        if row.neighborhood in list(med_rents.index):
            median_nb_rents.append(med_rents[row.sale_date[:4]][row.sale_date[5:]][row.neighborhood])
        else:
            median_nb_rents.append(0)
    return median_nb_rents


In [9]:
median_prices = get_med_prices(rent_estimate_df)
rent_estimate_df['median_nb_rents'] = median_prices

Runtime: 8.23 seconds, 65147 iterations remaining
Runtime: 16.08 seconds, 60147 iterations remaining
Runtime: 22.96 seconds, 55147 iterations remaining
Runtime: 31.54 seconds, 50147 iterations remaining
Runtime: 40.75 seconds, 45147 iterations remaining
Runtime: 44.8 seconds, 40147 iterations remaining
Runtime: 49.6 seconds, 35147 iterations remaining
Runtime: 57.75 seconds, 30147 iterations remaining
Runtime: 63.88 seconds, 25147 iterations remaining
Runtime: 71.3 seconds, 20147 iterations remaining
Runtime: 78.37 seconds, 15147 iterations remaining
Runtime: 86.87 seconds, 10147 iterations remaining
Runtime: 92.39 seconds, 5147 iterations remaining
Runtime: 97.92 seconds, 147 iterations remaining


In [10]:
rent_estimate_df.head()

,RegionID,neighborhood,address,sale_date,sale_price,zip_code,rent_estimate,median_nb_rents
0,270958,upper west side,360 MANHATTAN AVENUE,2012-05,695000.0,10026.0,2664,3312.0
1,270958,upper west side,360 MANHATTAN AVENUE,2012-03,665000.0,10026.0,2575,3200.0
2,270958,upper west side,360 MANHATTAN AVENUE,2012-02,721000.0,10026.0,2796,3399.0
3,270958,upper west side,360 MANHATTAN AVENUE,2013-08,955000.0,10026.0,3319,3250.0
4,270958,upper west side,130 MANHATTAN AVENUE,2012-12,2100000.0,10025.0,8155,3312.5


In [11]:
rent_estimate_df['above_median'] = np.where(rent_estimate_df.rent_estimate>rent_estimate_df.median_nb_rents,1,0)
# rent_estimate_df['above_median'] = np.where(rent_estimate_df.median_nb_rents==0,-1,rent_estimate_df.above_median)
# rent_estimate_df['above_median'] = np.where(rent_estimate_df.rent_estimate==rent_estimate_df.median_nb_rents,2,rent_estimate_df.above_median)

# Remove Extraneous Subway Stations

Remove subway station if there is a closer one that offers the same lines

In [12]:
df2 = nearest_subways_df.loc[nearest_subways_df.groupby(by=["address",'line'])["dist_miles"].idxmin()]
len(df2)

25776

In [13]:
df2 = df2.reset_index(drop=True)

In [14]:
#recalculate number of subways within distance
num_subs=[]
for a in df2.address:
    num_subs.append(len(df2[df2.address==a]))
    
df2.num_within_dist = num_subs
df2 = df2.reset_index(drop=True)

In [15]:
df2.head()

,address,num_within_dist,station,line,geometry,stat_lng,stat_lat,add_lng,add_lat,dist_miles
0,0 ALLEN STREET,5,Spring St,4-6-6 Express,POINT (-73.997141 40.72230089999999),-73.997141,40.722301,-73.990465,40.718715,0.428520
1,0 ALLEN STREET,5,Grand St,B-D,POINT (-73.9937529 40.7182669),-73.993753,40.718267,-73.990465,40.718715,0.174982
2,0 ALLEN STREET,5,Delancey St - Essex St,F,POINT (-73.988078 40.7186807),-73.988078,40.718681,-73.990465,40.718715,0.125005
3,0 ALLEN STREET,5,Delancey St - Essex St,J-M-Z,POINT (-73.9874094 40.718306),-73.987409,40.718306,-73.990465,40.718715,0.162479
4,0 ALLEN STREET,5,Bowery,J-Z,POINT (-73.9938069 40.72024688),-73.993807,40.720247,-73.990465,40.718715,0.204543


# Merge the dataframes

In [16]:
df = rent_estimate_df.merge(df2,on='address')

In [17]:
df[['RegionID', 'neighborhood', 'address',
    'sale_date','zip_code', 'station', 'line']] = df[['RegionID', 'neighborhood', 'address', 'sale_date',
                                                      'zip_code', 'station', 'line']].astype('category')

In [18]:
df.describe()

,sale_price,rent_estimate,median_nb_rents,above_median,num_within_dist,stat_lng,stat_lat,add_lng,add_lat,dist_miles
count,1.925740e+05,1.925740e+05,192574.000000,192574.000000,192574.000000,192574.000000,192574.000000,192574.000000,192574.000000,192574.000000
mean,2.980112e+06,1.009571e+04,3070.778418,0.697924,5.685794,-73.982541,40.735834,-73.982507,40.735880,0.287475
std,2.480936e+07,7.776543e+04,1367.820301,0.459159,2.924813,0.020642,0.041491,0.021058,0.041451,0.121341
min,1.001980e+05,2.750000e+02,0.000000,0.000000,1.000000,-74.030876,40.576034,-74.038328,40.574761,0.006265
25%,7.200000e+05,2.602000e+03,3000.000000,0.000000,3.000000,-73.998841,40.711835,-73.998682,40.709678,0.191397
50%,1.215000e+06,4.337000e+03,3450.000000,1.000000,6.000000,-73.982209,40.740893,-73.983784,40.740929,0.289731
75%,2.399801e+06,8.257000e+03,3700.000000,1.000000,8.000000,-73.969072,40.762970,-73.969458,40.764249,0.384712
max,2.200000e+09,6.772565e+06,6395.000000,1.000000,13.000000,-73.761353,40.874561,-73.767615,40.877360,0.551742


In [19]:
df = df[df.rent_estimate<=10000]

In [20]:
df.describe()

,sale_price,rent_estimate,median_nb_rents,above_median,num_within_dist,stat_lng,stat_lat,add_lng,add_lat,dist_miles
count,1.549620e+05,154962.000000,154962.000000,154962.000000,154962.000000,154962.000000,154962.000000,154962.000000,154962.000000,154962.000000
mean,1.160854e+06,4052.839232,2990.185149,0.624605,5.434971,-73.981170,40.734332,-73.981099,40.734271,0.288893
std,7.032479e+05,2282.890289,1401.057966,0.484226,2.833220,0.021253,0.044340,0.021664,0.044314,0.120948
min,1.001980e+05,275.000000,0.000000,0.000000,1.000000,-74.030876,40.576034,-74.038328,40.574761,0.006265
25%,6.460000e+05,2343.000000,2900.000000,0.000000,3.000000,-73.996209,40.707745,-73.995883,40.707984,0.193521
50%,9.750000e+05,3506.000000,3400.000000,1.000000,5.000000,-73.981963,40.740893,-73.982939,40.741389,0.291872
75%,1.565000e+06,5505.000000,3700.000000,1.000000,8.000000,-73.968379,40.762970,-73.967897,40.764217,0.385036
max,4.643500e+06,10000.000000,6395.000000,1.000000,13.000000,-73.761353,40.874561,-73.767615,40.877360,0.551742


In [21]:
# df_dum = df.copy()
# df_dum = pd.get_dummies(df_dum, columns=['station','line'])
# df_dum['round_distance'] = df_dum['dist_miles']*10
# df_dum['round_distance'] = df_dum['round_distance'].astype('int64')
# df_dum = pd.get_dummies(df_dum, columns=['round_distance'])

In [22]:
df_dummies = df.copy()
station_dummies = pd.get_dummies(df_dummies, columns=['station'])
line_dummies = station_dummies.line.str.split('-', expand=True).stack()
line_dum = pd.get_dummies(line_dummies, prefix='line').groupby(level=0).sum()
df_dummies = pd.merge(station_dummies,line_dum, on=line_dum.index)
df_dummies['round_distance'] = df_dummies['dist_miles']*10
df_dummies['round_distance'] = df_dummies['round_distance'].astype('int64')
df_dummies2 = pd.get_dummies(df_dummies, columns=['round_distance'])

In [23]:
df_dummies2 = df_dummies2.reset_index(drop=True).drop(columns=['key_0','line'])

In [29]:
df_dummies2.to_csv('../datasets/df_dummies2.csv')

# FIGURE THIS OUT!!!

### TRYING TO FLATTEN DATAFRAME SO THAT EVERY APT SALE IS ONE ROW INSTEAD OF NUM_WITHIN_DIST ROWS

In [52]:
test_dum = df_dummies2[0:3].copy()
test_dum.num_within_dist.max()

3

In [55]:
for index, row in test_dum.iterrows():
    print(index+row.num_within_dist)

3
4
5


In [ ]:
#test_dum.groupby(by=['RegionID','sale_date','rent_estimate','address']).max()

In [37]:
df_dummies2.head()

,RegionID,neighborhood,address,sale_date,sale_price,zip_code,rent_estimate,median_nb_rents,above_median,num_within_dist,geometry,stat_lng,stat_lat,add_lng,add_lat,dist_miles,station_103rd St,station_110th St,station_116th St,station_116th St - Columbia University,station_125th St,station_135th St,station_137th St - City College,station_138th St - Grand Concourse,station_145th St,station_149th St - Grand Concourse,station_14th St,station_155th St,station_157th St,station_15th St - Prospect Park,station_163rd St - Amsterdam Av,station_168th St,station_170th St,station_175th St,station_181st St,station_182nd-183rd Sts,station_18th Ave,...,station_Whitehall St,station_Wilson Ave,station_Winthrop St,station_Woodhaven Blvd,station_World Trade Center,station_York St,line_1,line_2,line_3,line_4,line_5,line_6,line_6 Express,line_7,line_7 Express,line_A,line_B,line_C,line_D,line_E,line_F,line_G,line_J,line_L,line_M,line_N,line_Q,line_R,line_S,line_W,line_Z,round_distance_0,round_distance_1,round_distance_2,round_distance_3,round_distance_4,round_distance_5
0,270958,upper west side,360 MANHATTAN AVENUE,2012-05,695000.0,10026.0,2664,3312.0,0,3,POINT (-73.96410990000001 40.807722),-73.964110,40.807722,-73.957293,40.804399,0.424107,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,270958,upper west side,360 MANHATTAN AVENUE,2012-05,695000.0,10026.0,2664,3312.0,0,3,POINT (-73.95182199999999 40.7990749),-73.951822,40.799075,-73.957293,40.804399,0.466040,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,270958,upper west side,360 MANHATTAN AVENUE,2012-05,695000.0,10026.0,2664,3312.0,0,3,POINT (-73.9547977 40.8050581),-73.954798,40.805058,-73.957293,40.804399,0.138218,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,270958,upper west side,360 MANHATTAN AVENUE,2012-03,665000.0,10026.0,2575,3200.0,0,3,POINT (-73.96410990000001 40.807722),-73.964110,40.807722,-73.957293,40.804399,0.424107,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,270958,upper west side,360 MANHATTAN AVENUE,2012-03,665000.0,10026.0,2575,3200.0,0,3,POINT (-73.95182199999999 40.7990749),-73.951822,40.799075,-73.957293,40.804399,0.466040,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [396]:
df_dum.to_csv('../datasets/df_dummy.csv')

In [36]:
df.address.unique()

[360 MANHATTAN AVENUE, 60 WEST 106 STREET, 60 WEST 107 STREET, 20 WEST 97TH STREET, 380 CENTRAL PARK WEST, ..., 2020 EAST 69TH STREET, 1300 EAST 69TH STREET, 10 VANDERBILT AVENUE, 0 VANDERBILT AVENUE, 20 VANDERBILT AVENUE]
Length: 7069
Categories (7069, object): [360 MANHATTAN AVENUE, 60 WEST 106 STREET, 60 WEST 107 STREET, 20 WEST 97TH STREET, ..., 1300 EAST 69TH STREET, 10 VANDERBILT AVENUE, 0 VANDERBILT AVENUE, 20 VANDERBILT AVENUE]